#Instalação de Pacotes

In [ ]:
!pip install -q -U google-generativeai
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB

#Importação de Bibliotecas

In [ ]:
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import gradio as gr

#Função para Formatar Texto como Markdown

In [ ]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Configuração da API do Google Generative AI

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

#Montagem do Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Caminho do Arquivo Excel

In [ ]:
PATH = '/content/drive/MyDrive/Projeto Final Cloud/Planilha.xlsx'

#Leitura da Planilha

In [ ]:
import pandas as pd
df = pd.read_excel(PATH)
df.head()

,Sintomas,Remédio,Valor
0,Dor de cabeça,"Paracetamol, Ibuprofeno","R$ 5,00 - R$ 20,00"
1,Dor de garganta,"Ibuprofeno, Pastilhas para garganta","R$ 5,00 - R$ 30,00"
2,Febre,"Paracetamol, Ibuprofeno","R$ 5,00 - R$ 20,00"
3,Coriza,"Descongestionante nasal, Soro fisiológico","R$ 10,00 - R$ 50,00"
4,Tosse,"Expectorante, Xarope para tosse","R$ 15,00 - R$ 60,00"


#Geração de Embeddings

In [ ]:
# Importação de bibliotecas adicionais para processamento de texto e cálculo de similaridade
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Função para gerar embeddings para um texto usando a API do Google Generative AI
def gerar_embeddings(texto):
    result = genai.embed_content(
        model="models/embedding-001",
        content=texto,
        task_type="retrieval_document",
        title="Embedding de um sintoma")
    return result['embedding']

# Geração de embeddings para todos os sintomas na planilha
df['Embeddings'] = df['Sintomas'].apply(lambda x: gerar_embeddings(x))
df.head()

,Sintomas,Remédio,Valor,Embeddings
0,Dor de cabeça,"Paracetamol, Ibuprofeno","R$ 5,00 - R$ 20,00","[0.040010504, -0.041635323, -0.0175412, -0.003..."
1,Dor de garganta,"Ibuprofeno, Pastilhas para garganta","R$ 5,00 - R$ 30,00","[0.03561967, -0.004000087, -0.027008975, -0.01..."
2,Febre,"Paracetamol, Ibuprofeno","R$ 5,00 - R$ 20,00","[0.017560635, 0.003794943, -0.02327602, -0.012..."
3,Coriza,"Descongestionante nasal, Soro fisiológico","R$ 10,00 - R$ 50,00","[0.06320564, -0.0073769316, -0.034813635, -0.0..."
4,Tosse,"Expectorante, Xarope para tosse","R$ 15,00 - R$ 60,00","[0.05174097, -0.025533872, -0.02973329, -0.004..."


#Função para Recomendação de Remédios

In [ ]:
# Função para recomendar remédio com base na similaridade de cosseno dos embeddings
def recomendar_remedio(sintoma_paciente):
    # Gera embedding para o sintoma do paciente
    embedding_paciente = gerar_embeddings(sintoma_paciente)
    embedding_paciente = np.array(embedding_paciente).reshape(1, -1)

    # Lista para armazenar as similaridades
    similaridades = []

    # Calcula a similaridade de cosseno entre o sintoma do paciente e cada sintoma na base de dados
    for embedding in df['Embeddings']:
        embedding = np.array(embedding).reshape(1, -1)
        similaridade = cosine_similarity(embedding_paciente, embedding)[0][0]
        similaridades.append(similaridade)

    # Converte a lista de similaridades em uma série do pandas
    similaridades = pd.Series(similaridades)

    # Encontra o índice da maior similaridade
    idx_max_similaridade = similaridades.idxmax()

    # Retorna o remédio e o valor mais similares
    recomendacao = {
        "Remedio": df.loc[idx_max_similaridade, "Remédio"],
        "Valor": df.loc[idx_max_similaridade, "Valor"]
    }

    return recomendacao

#------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
# Coleta a entrada dos sintomas do usuário
sintoma_novo_paciente = input("Por favor, descreva os seus sintomas: ")

# Chama a função para recomendar remédio com base nos sintomas fornecidos pelo usuário
resultado = recomendar_remedio(sintoma_novo_paciente)

# Exibe a recomendação de remédio
print(f"É recomendado o uso de {resultado['Remedio']}, o seu valor é {resultado['Valor']}")

model = genai.GenerativeModel('gemini-pro')

questao = f"Faça um breve resumo sobre o seguinte remédio {resultado['Remedio']}, fale também de quanto em quanto tempo é para tomar, disponibilize um link para pessoa comprar o remedio"
response = model.generate_content(questao)
to_markdown(response.text)

Por favor, descreva os seus sintomas: Estou com dor na garganta
É recomendado o uso de Ibuprofeno, Pastilhas para garganta, o seu valor é R$ 5,00 - R$ 30,00


> **Ibuprofeno, Pastilhas para Garganta**
> 
> Ibuprofeno é um medicamento anti-inflamatório não esteroide (AINE) usado para aliviar a dor, febre e inflamação.
> 
> **Indicações:**
> 
> * Dor de garganta
> * Tosse
> * Rouquidão
> 
> **Dosagem:**
> 
> * Adultos e crianças com mais de 12 anos: Tomar uma pastilha a cada 4-6 horas, conforme necessário.
> * Não exceder 6 pastilhas por dia.
> 
> **Duração do Tratamento:**
> 
> * Use o medicamento apenas enquanto os sintomas persistirem.
> * Não use por mais de 3 dias sem aconselhamento médico.
> 
> **Link para Comprar:**
> 
> https://www.drogariaonlinedrogariasp.com.br/produtos/ibuprofeno-pastilhas-600mg-herbarium-bucofaringe

In [ ]:
# Coleta a entrada dos sintomas do usuário
#sintoma_novo_paciente = input("Por favor, descreva os seus sintomas: ")

# Chama a função para recomendar remédio com base nos sintomas fornecidos pelo usuário
#resultado, encontrado = recomendar_remedio(sintoma_novo_paciente)

#if encontrado:
    # Exibe a recomendação de remédio
    #print(f"É recomendado o uso de {resultado['Remedio']}, o seu valor é {resultado['Valor']}")

    # Configura o modelo generativo
    #model = genai.GenerativeModel('gemini-pro')

    # Gera o resumo sobre o remédio recomendado
    #questao = f"Faça um breve resumo sobre o seguinte remédio {resultado['Remedio']}, fale também de quanto em quanto tempo é para tomar"
   # response = model.generate_content(questao)

    # Exibe o resumo gerado
    #display(to_markdown(response.text))
#else:
    # Se a similaridade não for suficiente, gera informações sobre o sintoma
    #print(f"Não encontramos um remédio específico para os sintomas fornecidos.")

    # Configura o modelo generativo
    #model = genai.GenerativeModel('gemini-pro')

    # Gera informações sobre o sintoma fornecido
    #questao = f"Fale sobre o seguinte sintoma: {sintoma_novo_paciente}"
    #response = model.generate_content(questao)

    # Exibe o resumo gerado
    #display(to_markdown(response.text))

In [ ]:
# Função para gerar um resumo do remédio
#def gerar_resumo_remedio(remedio):
    #model = genai.GenerativeModel('gemini-pro')
    #questao = f"Faça um breve resumo sobre o seguinte remédio {remedio}, fale também de quanto em quanto tempo é para tomar"
    #response = model.generate_content(questao)
    #return response.text

# Função combinada para a interface gráfica
#def obter_recomendacao(sintomas):
    #recomendacao = recomendar_remedio(sintomas)
    #remedio = recomendacao['Remedio']
    #valor = recomendacao['Valor']
    #resumo = gerar_resumo_remedio(remedio)
    #return f"Recomendação: {remedio}\nValor: {valor}\n\nResumo:\n{resumo}"

# Interface gráfica usando Gradio
#iface = gr.Interface(
    #fn=obter_recomendacao,
    #inputs=gr.Textbox(lines=5, label="Descreva os seus sintomas"),
    #outputs="text",
    #title="Recomendação de Remédio",
    #description="Digite seus sintomas e receba uma recomendação de remédio com informações sobre o uso.",
    #theme="compact"
#)

# Lançar a interface
#iface.launch()